In [2]:
import tensorflow as tf

2025-07-07 13:35:33.286364: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim):
        super(MyDenseLayer, self).__init__()

        self.w = self.add_weight([input_dim, output_dim])
        self.b = self.add_weight([1, output_dim])

    def call(self, inputs):
        z = tf.matmul(inputs, self.w) + self.b
        return tf.math.sigmoid(z)

In [5]:
input_dim = 3
output_dim = 2
layer = MyDenseLayer(input_dim, output_dim)
inputs = tf.constant([[1.0, 2.0, 3.0]])
outputs = layer(inputs)
print(outputs.numpy())

[[0.8867185 0.9546766]]


In [10]:
class SequentialModel(tf.keras.Model):
    def __init__(self, layers):
        super(SequentialModel, self).__init__()
        self.model_layers = layers

    def call(self, inputs):
        x = inputs 
        for layer in self.model_layers:
            x = layer(x)
        return x

In [13]:
layers = []
layers.append(MyDenseLayer(3, 4))
layers.append(MyDenseLayer(4, 3))
model = SequentialModel(layers)
inputs = tf.constant([[1.0, 2.0, 3.0]])
outputs = model(inputs)
print(outputs.numpy())

[[0.70461786 0.30614072 0.6488198 ]]
